# Statefarm Distracted Driver Classification (using Keras)
## Satchel Grant

The goal of this notebook is to classify the statefarm distracted drivers using Keras instead of TensorFlow. I also will implement a generator for data feeding to reduce the memory consumption.

### Initial Imports

In [1]:
import numpy as np
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
from sklearn.utils import shuffle
import pickle
import scipy.misc as sci
import time
import sys

%matplotlib inline

def show_img(img):
    plt.imshow(img)
    plt.show()

### Read in Data

First I read in the file paths of each of the image files and create a parallel array to store the labels and then I shuffle the order of the data.


In [2]:
external_drive_path = '/Volumes/WhiteElephant/'
home_path = os.getcwd()
os.chdir(external_drive_path)

In [3]:
path = './statefarm_drivers/imgs/train'

def read_paths(path, no_labels=False):
    file_paths = []
    labels = []
    labels_to_nums = dict()
    for dir_name, subdir_list, file_list in os.walk(path):
        if len(subdir_list) > 0:
            label_types = subdir_list
            for i,subdir in enumerate(subdir_list):
                labels_to_nums[subdir] = i
        for img_file in file_list:
            if '.jpg' in img_file.lower():
                file_paths.append(os.path.join(dir_name,img_file))
                if no_labels: labels.append(img_file)
                else: labels.append(labels_to_nums[dir_name[-2:]])
    if no_labels: return file_paths, labels
    n_labels = len(label_types)
    return file_paths, labels, n_labels

file_paths, labels,n_labels = read_paths(path)
file_paths, labels = shuffle(file_paths, labels)
print("Number of data samples: " + str(len(file_paths)))

Number of data samples: 22424


In [4]:
def one_hot_encode(labels, n_classes):
    one_hots = []
    for label in labels:
        one_hot = [0]*n_classes
        one_hot[label] = 1
        one_hots.append(one_hot)
    return np.array(one_hots,dtype=np.float32)

labels = one_hot_encode(labels,n_labels)

The following cell contains some data augmentation functions to increase the amount of useable data. It includes rotations, translations, and combinations of the two.

In [5]:
def rotate(img, angle):
    rot_img = sci.imrotate(img, angle).astype(img.dtype)
    rand_filler = np.random.random(rot_img.size).astype(rot_img.dtype)
    rot_img[ones[:,:]!=1] = rand_filler[ones[:,:]!=1]
    return rot_img

def translate(img, row_amt, col_amt):
    translation = np.random.random(img.shape).astype(img.dtype)
    if row_amt > 0:
        if col_amt > 0:
            translation[row_amt:,col_amt:] = img[:-row_amt,:-col_amt]
        else:
            translation[row_amt:,:-col_amt] = img[:-row_amt,col_amt:]
    else:
        if col_amt > 0:
            translation[:-row_amt,col_amt:] = img[row_amt:,:-col_amt]
        else:
            translation[:-row_amt,:-col_amt] = img[row_amt:,col_amt:]
    return translation





Next I create a generator to read in the images in batches. This reduces the amount of memory required to deal with the entire dataset.

In [6]:
split_index = int(.75*len(file_paths))
X_train_paths, y_train = file_paths[:split_index], labels[:split_index]
X_valid_paths, y_valid = file_paths[split_index:], labels[split_index:]
batch_size = 128
train_steps_per_epoch = len(X_train_paths)//batch_size + 1
if len(X_train_paths) % batch_size == 0: train_steps_per_epoch = len(X_train_paths)//batch_size
valid_steps = len(X_valid_paths)//batch_size
resize_dims = (120,120)


def convert_images(paths, resize_dims, img_depth=3):
    images = np.empty((len(paths),resize_dims[0],resize_dims[1],img_depth),dtype=np.float32)
    for i,path in enumerate(paths):
        img = mpimg.imread(path)
        images[i] = sci.imresize(img, resize_dims)
    return images

def image_generator(file_paths, labels, batch_size, resize_dims=(120,120),testing=False,img_depth=3):
    while 1:
        for batch in range(0, len(file_paths), batch_size):
            images = convert_images(file_paths[batch:batch+batch_size],resize_dims,img_depth=img_depth)
            batch_labels = labels[batch:batch+batch_size]
            if testing: yield images
            else: yield images, batch_labels


train_generator = image_generator(X_train_paths, y_train, batch_size)
valid_generator = image_generator(X_valid_paths, y_valid, len(X_valid_paths))

In [ ]:
X_valid_batch,y_valid_batch = next(valid_generator)

### Keras Imports

In [7]:
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Dense, Input, concatenate, \
        Flatten, Dropout, Lambda
from keras.layers.normalization import BatchNormalization


Using TensorFlow backend.


### Model Architecture

The model consists of 4 convolutional stacks followed by 2 dense layers. I had good success with this model while predicting the required steering angle from an image of a track for a car to drive around a track in real time. It is also a lightweight model making it quick and easy to train.


In [8]:
stacks = []
conv_shapes = [(1,1),(3,3),(5,5)]
conv_depths = [8,10,10,10]
pooling_filter = (2,2)
pooling_stride = (2,2)
dense_shapes = [150,50,n_labels]

inputs = Input(shape=(resize_dims[0],resize_dims[1],3))
zen_layer = BatchNormalization()(inputs)

for shape in conv_shapes:
    stacks.append(Conv2D(conv_depths[0], shape, padding='same', activation='elu')(inputs))
layer = concatenate(stacks,axis=-1)
layer = BatchNormalization()(layer)
layer = MaxPooling2D(pooling_filter,strides=pooling_stride,padding='same')(layer)
layer = Dropout(0.05)(layer)

for i in range(1,len(conv_depths)):
    stacks = []
    for shape in conv_shapes:
        stacks.append(Conv2D(conv_depths[i],shape,padding='same',activation='elu')(layer))
    layer = concatenate(stacks,axis=-1)
    layer = BatchNormalization()(layer)
    layer = MaxPooling2D(pooling_filter,strides=pooling_stride, padding='same')(layer)

layer = Flatten()(layer)
layer = Dropout(0.5)(layer)

for i in range(len(dense_shapes)-1):
    layer = Dense(dense_shapes[i], activation='elu')(layer)
    layer = BatchNormalization()(layer)

outputs = Dense(dense_shapes[-1], activation='softmax')(layer)

### Training and Validation
The next cell trains the model using the adam optimizer and categorical_crossentropy. The adam optimizer is most efficient because it has specific learning rates for each parameter in the net and it uses momentum. Both of these techniques improves the efficiency of the training process.

I use the categorical_crossentropy loss function because this a good loss function for classification problems.

In [9]:
model = Model(inputs=inputs,outputs=outputs)
model.load_weights('model.h5')
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# score = model.evaluate_generator(X_valid_batch, y_valid_batch, batch_size=128, verbose=0)
# print(score)


### Testing

In [10]:
path = './statefarm_drivers/imgs/test'
test_paths,test_labels = read_paths(path,no_labels=True)
print(str(len(test_paths))+' testing images')

79726 testing images


In [11]:
del file_paths
del labels

In [11]:
test_divisions = 4
test_generator = image_generator(test_paths,test_labels,len(test_paths)//test_divisions+1,testing=True)

In [12]:
predictions = []
batch_size = 1500

In [ ]:
counter = 1

In [ ]:
counter+=1
base_time = time.time()

print('Begin set ' + str(2))
X_set = next(test_generator)
print('Running time: ' + str(time.time()-base_time)+ 's')

n_batches = len(X_set)//batch_size+1
if len(X_set)%batch_size == 0: n_batches = len(X_set)//batch_size
print('N_batches: '+ str(n_batches))

sys.stdout.write(' '*n_batches+'|')
for batch in range(0,len(X_set),batch_size):
    predictions.append(model.predict_on_batch(X_set[batch:batch+batch_size]))
    sys.stdout.write("*")

print("\nExecution Time: " + str((time.time()-base_time)/60)+'min')

Begin set 1


In [16]:
print(len(predictions))

41


In [17]:
base_time = time.time()
# setup toolbar
sys.stdout.write("[%s]" % (" " * n_batches))
sys.stdout.flush()
sys.stdout.write("\b" * (n_batches+1)) # return to start of line, after '['


for batch in range(1,n_batches):
    predictions.append(model.predict_on_batch(X_set[batch:batch+batch_size]))
    sys.stdout.write("-")
    sys.stdout.flush()

print("\nExecution Time: " + str((time.time()-base_time)/60)+'min')

[                    ]-------------------
Execution Time: 13.875528434912363min


In [19]:
base_time = time.time()
print('Begin set ' + str(4))
X_set = next(test_generator)
print('Running time: ' + str(time.time()-base_time)+ 's')
n_batches = len(X_set)//batch_size+1
if len(X_set)%batch_size == 0: n_batches = len(X_set)//batch_size
print('N_batches: '+ str(n_batches))


sys.stdout.write("[%s]" % (" " * n_batches))
sys.stdout.flush()
sys.stdout.write("\n")

for batch in range(n_batches):
    predictions.append(model.predict_on_batch(X_set[batch:batch+batch_size]))
    sys.stdout.write("-")
    sys.stdout.flush()

print("\nExecution Time: " + str((time.time()-base_time)/60)+'min')

Begin set 4
Running time: 356.13717818260193s
N_batches: 20
[                    ]--------------------
Execution Time: 19.960290618737538min


In [22]:
for p in predictions:
    
    print(len(p))

1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000


In [21]:

with open('./statefarm_drivers/submission.csv', 'w') as f:
    f.write('img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9\n')
    for i,logit_group in enumerate(predictions):
        for j,logit in enumerate(logit_group):
            id_ = test_labels[i*len(logit_group)+j]
            f.write(id_+',')
            for k,element in enumerate(logit):
                if k == logit.shape[0]-1: f.write(str(element)+'\n')
                else: f.write(str(element)+',')

IndexError: list index out of range